In [172]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [155]:
x = pd.read_csv('C:/Users/ADMIN/Desktop/Code/Jupyter N_book/Week 3/training_set_features.csv')
y = pd.read_csv('C:/Users/ADMIN/Desktop/Code/Jupyter N_book/Week 3/training_set_labels.csv')
x_train = x.dropna()

for i in y['respondent_id']:
    if i not in x_train['respondent_id']:
        y.drop(i,axis = 0, inplace = True)

xyz1 = x_train.filter(like = 'xyz')
seas1 = x_train.filter(like = 'seas')
temp_int = x_train.select_dtypes(include = [int,float])
xyz_int = temp_int.copy()
for col in seas1:
    xyz_int.drop(col, axis=1, inplace=True)

seas_int = temp_int.copy()
for col in xyz1:
    seas_int.drop(col, axis=1, inplace=True)

y.shape

(6437, 3)

In [156]:
temp_category = x_train.select_dtypes(include = [object])
temp_cat = temp_category.astype('category')
lab_code = LabelEncoder()
for col in temp_cat:
    temp_cat[col] = lab_code.fit_transform(temp_cat[col])
temp_cat.head()

,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,employment_industry,employment_occupation
1,1,0,3,1,2,1,1,0,1,0,12,19
7,2,3,3,0,0,0,0,0,1,2,12,20
10,2,0,3,1,0,0,1,0,6,0,19,13
11,3,2,3,1,1,0,0,0,8,2,19,1
15,1,2,3,0,1,0,0,0,6,0,18,5


In [164]:
xyz_train = xyz_int.join(temp_cat)
seas_train = seas_int.join(temp_cat)
xyz_train.drop('respondent_id', axis = 1,inplace = True)
seas_train.drop('respondent_id', axis = 1,inplace = True)

minmax = MinMaxScaler()
xyz_col = xyz_train.columns
xyz_train= pd.DataFrame(minmax.fit_transform(xyz_train),columns = xyz_col)
seas_col = seas_train.columns
seas_train= pd.DataFrame(minmax.fit_transform(seas_train),columns = seas_col)


x_train,x_test,y_train,y_test = train_test_split(xyz_train,y.iloc[:,1],test_size=0.33)
sx_train,sx_test,sy_train,sy_test = train_test_split(seas_train,y.iloc[:,2],test_size=0.33)

In [173]:
cross_val_score(LogisticRegression(max_iter=1000),x_train,y_train)

array([0.81807648, 0.82155272, 0.81902552, 0.83410673, 0.82830626])

In [174]:
cross_val_score(svm.SVC(kernel= 'rbf',max_iter=1000),x_train,y_train)

array([0.81691773, 0.80648899, 0.80278422, 0.83294664, 0.82134571])

In [175]:
cross_val_score(svm.SVC(kernel= 'poly',max_iter=1000),x_train,y_train)

array([0.73348783, 0.76825029, 0.76914153, 0.77146172, 0.73317865])

In [176]:
met = [LogisticRegression(max_iter=1000),svm.SVC(kernel= 'rbf',max_iter=1000)]
for i in met:
    print(f' The score is for {i} is: {cross_val_score(i,x_test,y_test)}')

 The score is for LogisticRegression(max_iter=1000) is: [0.82823529 0.83529412 0.83058824 0.81882353 0.8       ]
 The score is for SVC(max_iter=1000) is: [0.83058824 0.82588235 0.82588235 0.80470588 0.80470588]


In [178]:
##Making Pipelines

In [180]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [243]:
x = pd.read_csv('C:/Users/ADMIN/Desktop/Code/Jupyter N_book/Week 3/training_set_features.csv')
y = pd.read_csv('C:/Users/ADMIN/Desktop/Code/Jupyter N_book/Week 3/training_set_labels.csv')
xy = pd.merge(x,y,on = 'respondent_id')
xy.dropna()
target_xyz = xy['xyz_vaccine']
target_seas = xy['seasonal_vaccine']

x_train,x_test,y_train,y_test = train_test_split(xy,target_xyz,test_size=0.33)

In [244]:
num_feature_xyz = xy.select_dtypes(include = ['int','float'])
num_feature_xyz_drop = num_feature_xyz.drop(set(xy.filter(regex='seas').columns) | {'respondent_id', 'seasonal_vaccine'}, axis = 1, inplace = True)
num_feature_seas = xy.select_dtypes(include = ['int','float'])
nums_features_seas_drop = num_feature_seas.drop(set(xy.filter(regex='xyz').columns) | {'respondent_id', 'xyz_vaccine'}, axis = 1, inplace = True)
num_stand = Pipeline(steps=[('scaler',MinMaxScaler())])

obj_features = xy.select_dtypes(include = ['object'])
cat_features = obj_features.astype('category')
cat_stand = Pipeline(steps=[('Encode',LabelEncoder())])

preprocess_xyz = ColumnTransformer(transformers=[('nums',num_stand,num_feature_xyz_drop,num_feature_xyz),('cat',cat_stand,cat_features,obj_features)])

preprocess_seas = ColumnTransformer(transformers=[('nums',num_stand,nums_features_seas_drop,num_feature_seas),('cat',cat_stand,cat_features,obj_features)])


pipeline_xyz = Pipeline(steps = [('preprocess_xyz',preprocess_xyz), ('SVM',svm.SVC())])
pipeline_seas = Pipeline(steps = [('preprocess_seas',preprocess_seas), ('SVM',svm.SVC())])


In [245]:
model = pipeline_xyz.fit(x_train,x_test)

ValueError: too many values to unpack (expected 3)

In [181]:
TEST = pd.read_csv('C:/Users/ADMIN/Desktop/Code/Jupyter N_book/Week 3/test_set_features.csv')
TEST.shape

(26708, 36)